In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
os.getcwd()

'C:\\Users\\olubl\\OneDrive\\College\\Data+\\DPS-Planning'

In [82]:
#loading in the current enrollment for only the 2024-25 school year
current_enrollment = pd.read_csv(r'C:\\Users\\olubl\\OneDrive\\College\\Data+\\DPS-Planning\DPS_count_pu_grades.csv')
current_enrollment_2024 = current_enrollment[current_enrollment['year_str']=='2024-2025']
current_enrollment_2024.head()
current_enrollment['geo_id_b2020'].unique()

array([3.7063e+14, 3.7064e+14,        nan])

In [91]:
#now we want to only show the 5th, 6th, 7th, and 8th grade - the grades that will be in high school in 2028-29
mask = current_enrollment['grade'].isin([5,6,7,8])
hs_2028 = current_enrollment_2024[mask].dropna()
hs_2028 = hs_2028.drop(columns=['fall_year','spring_year','year_str','geo_id_b2020'])
hs_2028.head(10)

C:\Users\olubl\AppData\Local\Temp\ipykernel_2804\3691576906.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hs_2028 = current_enrollment_2024[mask].dropna()


,geo_nm_b2020,grade,count,basez
19,Block 1000,5,2.0,1.0
22,Block 1000,6,1.0,1.0
25,Block 1000,7,1.0,1.0
29,Block 1000,8,1.0,0.0
51,Block 1001,8,2.0,2.0
73,Block 1002,5,1.0,1.0
77,Block 1002,8,1.0,0.0
93,Block 1003,6,2.0,1.0
95,Block 1003,7,1.0,1.0
97,Block 1003,8,1.0,1.0


In [98]:
#merging identical rows to get student counts by block and grade
columns_to_group = (['geo_nm_b2020','grade'])
hs_2028 = hs_2028.groupby(columns_to_group,as_index=False).sum()   #as_index=False preserves the columns we are grouping by
ms = {5,6,7,8}
hs_2028[hs_2028['grade'].isin(ms)]['count'].sum()  #there are this many dps students in 5th-8th grade now

8775.0

In [99]:
hs_2028.head()
#now use marketshare for projections

,geo_nm_b2020,grade,count,basez
0,Block 1000,5,70.0,44.0
1,Block 1000,6,55.0,35.0
2,Block 1000,7,50.0,28.0
3,Block 1000,8,64.0,36.0
4,Block 1001,5,55.0,35.0


In [44]:
#read in the planning units
dps_pu = gpd.read_file(r'C:\\Users\\olubl\\OneDrive\\College\\Data+\\DPS-Planning\PU_2425_918\PU_2425_918.shp')
dps_pu

,OBJECTID,pu_2324_84,X,Y,TIMS_PU,pu_2425_91,Join_Count,TARGET_FID,Join_Cou_1,TARGET_F_1,...,sch_id6__2,sch_id_g_2,choice_hs,type_hs,region_202,region_2_1,pu_2425__2,Shape_Leng,Shape_Area,geometry
0,1.0,1,2.049028e+06,893128.606346,PU589,1,1,1,1,1,...,320356,356,None,High School Base Assignment Zone,North,North,820,26532.120021,3.556198e+07,"POLYGON ((2.05e+06 8.89e+05, 2.05e+06 8.89e+05..."
1,2.0,2,2.039923e+06,835434.121317,PU401,2,1,2,1,2,...,320356,356,None,High School Base Assignment Zone,North,North,664,19333.085511,2.080136e+07,"POLYGON ((2.04e+06 8.38e+05, 2.04e+06 8.37e+05..."
2,3.0,3,2.058005e+06,797649.028742,PU782.0,3,1,3,1,3,...,320368,368,Energy & Sustainability,High School Base Assignment Zone,East,East,1,18314.823119,1.329077e+07,"POLYGON ((2.06e+06 7.99e+05, 2.06e+06 7.98e+05..."
3,4.0,4,2.003222e+06,772895.737307,PU201.0,4,1,4,1,4,...,320312,312,None,High School Base Assignment Zone,Southwest,Southwest,404,29282.100083,4.684140e+07,"POLYGON ((2e+06 7.78e+05, 2e+06 7.78e+05, 2e+0..."
4,5.0,5,2.067334e+06,828244.230730,PU646,5,1,5,1,5,...,320368,368,Energy & Sustainability,High School Base Assignment Zone,East,East,133,60685.151474,1.497046e+08,"POLYGON ((2.07e+06 8.3e+05, 2.08e+06 8.28e+05,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,914.0,112,2.023281e+06,785449.629502,PU210,914,1,914,1,914,...,320325,325,International Baccalaureate (IB),High School Base Assignment Zone,Southeast,Southeast,258,3542.983124,7.631459e+05,"POLYGON ((2.02e+06 7.85e+05, 2.02e+06 7.85e+05..."
914,915.0,832,2.048169e+06,803230.840232,PU102,915,1,915,1,915,...,320368,368,Energy & Sustainability,High School Base Assignment Zone,East,East,62,5086.589208,1.255585e+06,"POLYGON ((2.05e+06 8.03e+05, 2.05e+06 8.02e+05..."
915,916.0,355,2.076482e+06,810581.607204,PU800,916,1,916,1,916,...,320368,368,Energy & Sustainability,High School Base Assignment Zone,East,East,63,8776.259545,2.104188e+06,"POLYGON ((2.08e+06 8.12e+05, 2.08e+06 8.12e+05..."
916,917.0,607,2.026400e+06,786936.630124,PU727.1,917,1,917,1,917,...,320325,325,International Baccalaureate (IB),High School Base Assignment Zone,Southeast,Southeast,284,10649.868851,3.143675e+06,"POLYGON ((2.03e+06 7.85e+05, 2.03e+06 7.85e+05..."
